This is an exploratory analysis of animal complaints in Brisbane. There are a few interesting things I'd like to show - animal complaints per suburb of course, but animal complaints per square kilometre and per person (by pulling suburb info as well) will be equally interesting. Trends over time - which suburbs have improved or gotten worse over a few quarters. This is all being done in JupyterLab in Python 3.7.6.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
apr_to_jun_19 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-apr-jun-2019.csv")

In [3]:
apr_to_jun_18 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-apr-to-jun-2018.csv")

In [4]:
jan_to_mar_19 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-jan-mar-2019.csv")

In [5]:
jan_to_mar_18 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-jan-to-mar-2018.csv")

In [6]:
jul_to_sep_18 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-jul-to-sep-2018.csv")

In [7]:
jul_to_sep_19 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-jul-to-sep-2019.csv")

In [8]:
oct_to_dec_18 = pd.read_csv("/home/jack/AnimalC/cars-bis-open-data-animal-related-complaints-oct-to-dec-2018.csv")

In [9]:
oct_to_dec_19 = pd.read_csv("/home/jack/AnimalC/cars-srsa-open-data-animal-related-complaints-oct-to-dec-2019.csv")

In [10]:
jan_to_mar_20 = pd.read_csv("/home/jack/AnimalC/cars-srsa-open-data-animal-related-complaints-jan-to-mar-2020.csv")

The first thing that needs doing is to make all this data one usable dataset. The data itself isn't dated, so the only way of knowing what date a complaint is from is by looking at the dataset it's from. 

In [11]:
apr_to_jun_19["Date"] = np.datetime64('2019-05-01')

For ease of charting later on a column is added to each dataframe with the date being the 1st of the middle month of the dataset in question. So Apr to June 19 has the date of each incident listed as 1 May 2019.

In [12]:
apr_to_jun_18["Date"] = np.datetime64('2018-05-01')
jan_to_mar_19["Date"] = np.datetime64('2019-02-01')
jan_to_mar_18["Date"] = np.datetime64('2018-02-01')
jul_to_sep_18["Date"] = np.datetime64('2018-08-01')
jul_to_sep_19["Date"] = np.datetime64('2019-08-01')
oct_to_dec_18["Date"] = np.datetime64('2018-11-01')
oct_to_dec_19["Date"] = np.datetime64('2019-11-01')
jan_to_mar_20["Date"] = np.datetime64('2020-02-01')

In [13]:
jan18_to_mar_20 = pd.concat([apr_to_jun_18, jan_to_mar_19, jan_to_mar_18, jul_to_sep_18, jul_to_sep_19, oct_to_dec_18, oct_to_dec_19, jan_to_mar_20])

In [14]:
jan18_to_mar_20

,Category: Nature,Category: Type,Category: Reporting Level,Location: Suburb,Date,Office: Responsible Office
0,Animal,Other Animal,Unregistered,MORNINGSIDE,2018-05-01,NaN
1,Animal,Other Animal,Fox,GUMDALE,2018-05-01,NaN
2,Animal,Cat,NaN,CARSELDINE,2018-05-01,NaN
3,Animal,Dog,Fencing Issues,ZILLMERE,2018-05-01,NaN
4,Animal,Dog,Odour,CAMP HILL,2018-05-01,NaN
...,...,...,...,...,...,...
2141,Animal,Attack,Attack On An Animal,ENOGGERA,2020-02-01,NaN
2142,Animal,Dog,Wandering,EVERTON PARK,2020-02-01,NaN
2143,Animal,Cat,Wandering,BRACKEN RIDGE,2020-02-01,NaN
2144,Animal,Dog,Fencing Issues,INALA,2020-02-01,NaN


In [15]:
# Responsible office only shows up in a single dataset here - not really useful for analysing them as a whole.
del jan18_to_mar_20['Office: Responsible Office']

In [16]:
jan18_to_mar_20

,Category: Nature,Category: Type,Category: Reporting Level,Location: Suburb,Date
0,Animal,Other Animal,Unregistered,MORNINGSIDE,2018-05-01
1,Animal,Other Animal,Fox,GUMDALE,2018-05-01
2,Animal,Cat,NaN,CARSELDINE,2018-05-01
3,Animal,Dog,Fencing Issues,ZILLMERE,2018-05-01
4,Animal,Dog,Odour,CAMP HILL,2018-05-01
...,...,...,...,...,...
2141,Animal,Attack,Attack On An Animal,ENOGGERA,2020-02-01
2142,Animal,Dog,Wandering,EVERTON PARK,2020-02-01
2143,Animal,Cat,Wandering,BRACKEN RIDGE,2020-02-01
2144,Animal,Dog,Fencing Issues,INALA,2020-02-01


One notable thing stands out already. While a few Category: Reporting Level pieces are missing, a look at the data shows that this is only sometimes an issue - sometimes it's just that the Type has no Reporting Level below it - Cat Trapping is one example. However, given that this data is primarily useful and interesting on a *suburb* level, the 200 or so rows without suburb data are almost completely useless.

In [17]:
# This gives a new CSV to deal with in future if we need to use this file elsewhere.
jan18_to_mar_20.to_csv("jan18_to_mar_20.csv", index=False)

In [18]:
# The index numbers repeat, but they're not too important for what we're looking to do.
jan18_to_mar_20

,Category: Nature,Category: Type,Category: Reporting Level,Location: Suburb,Date
0,Animal,Other Animal,Unregistered,MORNINGSIDE,2018-05-01
1,Animal,Other Animal,Fox,GUMDALE,2018-05-01
2,Animal,Cat,NaN,CARSELDINE,2018-05-01
3,Animal,Dog,Fencing Issues,ZILLMERE,2018-05-01
4,Animal,Dog,Odour,CAMP HILL,2018-05-01
...,...,...,...,...,...
2141,Animal,Attack,Attack On An Animal,ENOGGERA,2020-02-01
2142,Animal,Dog,Wandering,EVERTON PARK,2020-02-01
2143,Animal,Cat,Wandering,BRACKEN RIDGE,2020-02-01
2144,Animal,Dog,Fencing Issues,INALA,2020-02-01


In [19]:
jan18_to_mar_20.rename(columns={'Category: Nature' : 'Nature', 'Category: Type' : 'Type',
                               'Category: Reporting Level' : 'Reporting Level', 'Location: Suburb' : 'Suburb'}, inplace=True)

In [20]:
jan18_to_mar_20 = jan18_to_mar_20[jan18_to_mar_20["Suburb"].notnull()]

In [21]:
jan18_to_mar_20

,Nature,Type,Reporting Level,Suburb,Date
0,Animal,Other Animal,Unregistered,MORNINGSIDE,2018-05-01
1,Animal,Other Animal,Fox,GUMDALE,2018-05-01
2,Animal,Cat,NaN,CARSELDINE,2018-05-01
3,Animal,Dog,Fencing Issues,ZILLMERE,2018-05-01
4,Animal,Dog,Odour,CAMP HILL,2018-05-01
...,...,...,...,...,...
2141,Animal,Attack,Attack On An Animal,ENOGGERA,2020-02-01
2142,Animal,Dog,Wandering,EVERTON PARK,2020-02-01
2143,Animal,Cat,Wandering,BRACKEN RIDGE,2020-02-01
2144,Animal,Dog,Fencing Issues,INALA,2020-02-01


In [22]:
suburb_counts = jan18_to_mar_20['Suburb'].value_counts()
print(suburb_counts["NUNDAH"])
print(suburb_counts["FORTITUDE VALLEY"])
suburb_counts

109
18


INALA                 500
BRACKEN RIDGE         284
FOREST LAKE           271
WYNNUM                260
ACACIA RIDGE          229
                     ... 
LAKE MANCHESTER         3
BULWER                  3
PORT OF BRISBANE        3
ENOGGERA RESERVOIR      1
MORETON ISLAND          1
Name: Suburb, Length: 189, dtype: int64

This is where the exploratory analysis starts to show some cracks. There are several important factors in explaining animal complaints. Firstly of course is population - both animal and human. We have no way of getting information on the former, though, so we'll need to use the human population as a proxy for it. Other factors involve things like the density of the suburb - we can see above that Fortitude Valley has very few animal complaints, likely due to having few animals relative to the human population. Feral cats, wandering dogs, etc, are more likely to be seen in suburbia than in the CBD.

The first is the most important to address, though. Inala has had 500 complaints over this time period, and Nundah, a suburb of similar size, has had 109. But Forest Lake has had two and a half times the complaints, but also has twice the population. Without proportionality we are unlikely to get a picture of what suburbs suffer more animal complaints. After all, suburbs are an artificial boundary - Forest Lake has ten times the population and six times the area of Seven Hills. If we compare on a suburb-to-suburb basis we're not really getting actionable information.

The next step is going to be pulling census data from the ABS and using that to compare with the various suburbs.

In [23]:
# census_16 contains the population data for all suburbs in Queensland. However, these are listed by SSC code rather than suburb name. To match them with the suburb names
# in our current dataframe this needs to be updated.
census_16 = pd.read_csv("/home/jack/AnimalC/2016Census_G01_QLD_SSC.csv")
ssc_codes = pd.read_csv("/home/jack/AnimalC/SSC_2016_AUST.csv")

In [24]:
# The below presents an opportunity. With net area in km^2 given for sub-areas of suburbs, we can sum up suburb areas as well as populations.
qld_codes = ssc_codes[ssc_codes['STATE_NAME_2016'] == 'Queensland']
qld_codes.head()

,MB_CODE_2016,SSC_CODE_2016,SSC_NAME_2016,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM
194894,30486210000,30031,Alexandra Hills,3,Queensland,0.0048
194895,30486190000,30031,Alexandra Hills,3,Queensland,0.0295
194896,30590710000,30031,Alexandra Hills,3,Queensland,0.0009
194897,30487120000,30031,Alexandra Hills,3,Queensland,0.0877
194898,30487130000,30031,Alexandra Hills,3,Queensland,0.0789


In [25]:
# This feels like an ugly and inefficient way to solve this. At 2 am, though, beggars can't be choosers.
areadict = {}

for i in range(0, len(qld_codes)):
    if qld_codes.iloc[i]["SSC_NAME_2016"] in areadict:
        suburb = qld_codes.iloc[i]["SSC_NAME_2016"]
        areadict[suburb] += qld_codes.iloc[i]["AREA_ALBERS_SQKM"]
    else:
        suburb = qld_codes.iloc[i]["SSC_NAME_2016"]
        areadict[suburb] = qld_codes.iloc[i]["AREA_ALBERS_SQKM"]

In [26]:
# First series to be part of a new dataframe. Area sqkm per suburb.
area_series = pd.Series(areadict)

In [27]:
area_series.sort_values()

No usual address (Qld)                       0.0000
Migratory - Offshore - Shipping (Qld)        0.0000
Guijar Islet                                 0.0793
Green Island                                 0.1239
Burrar Islet                                 0.1371
                                            ...    
Adavale                                  21354.7571
Diamantina Lakes                         21713.2812
Eromanga                                 28213.2083
Birdsville                               32959.5201
Bedourie                                 36118.1024
Length: 3264, dtype: float64

In [28]:
ssc_dict = {}

for i in range(0, len(qld_codes)):
    if qld_codes.iloc[i]["SSC_NAME_2016"] not in ssc_dict:
        suburb = qld_codes.iloc[i]["SSC_NAME_2016"]
        ssc_dict[suburb] = qld_codes.iloc[i]["SSC_CODE_2016"]


In [29]:
# Second series to be part of a new dataframe. SSC codes per suburb.
ssc_series = pd.Series(ssc_dict)

In [30]:
ssc_series

Alexandra Hills                          30031
Wellington Point                         33057
Chandler (Qld)                           30583
Belmont (Qld)                            30226
Ransome                                  32416
                                         ...  
Poona                                    32370
Tuan                                     32905
Tinana                                   32855
Migratory - Offshore - Shipping (Qld)    39797
No usual address (Qld)                   39494
Length: 3264, dtype: int64

In [31]:
# Sorting by SSC_Code to make this line up with the census data.
codes_area = pd.concat([ssc_series, area_series], axis=1)
codes_area = codes_area.rename(columns={0: 'SSC_Code', 1: 'Area'})
codes_area = codes_area.sort_values(by=['SSC_Code'])
codes_area = codes_area.reset_index()

In [32]:
# This gives the new dataframe an index identical to the census one.
codes_area

,index,SSC_Code,Area
0,Abbeywood,30001,35.0348
1,Abbotsford (Qld),30002,85.7638
2,Abercorn,30003,94.2732
3,Abergowrie,30004,306.2652
4,Abingdon Downs,30005,5052.0282
...,...,...,...
3259,Yuruga,33260,108.4832
3260,Zillmere,33261,3.6568
3261,Zilzie,33262,11.6773
3262,No usual address (Qld),39494,0.0000


In [33]:
codes_area = codes_area.rename(columns={'index': 'Suburb'})

In [34]:
census_16["SSC_CODE_2016"] = census_16["SSC_CODE_2016"].str.replace('SSC','')

In [35]:
census_16

,SSC_CODE_2016,Tot_P_M,Tot_P_F,Tot_P_P,Age_0_4_yr_M,Age_0_4_yr_F,Age_0_4_yr_P,Age_5_14_yr_M,Age_5_14_yr_F,Age_5_14_yr_P,...,High_yr_schl_comp_Yr_8_belw_P,High_yr_schl_comp_D_n_g_sch_M,High_yr_schl_comp_D_n_g_sch_F,High_yr_schl_comp_D_n_g_sch_P,Count_psns_occ_priv_dwgs_M,Count_psns_occ_priv_dwgs_F,Count_psns_occ_priv_dwgs_P,Count_Persons_other_dwgs_M,Count_Persons_other_dwgs_F,Count_Persons_other_dwgs_P
0,30001,9,8,23,0,0,0,3,0,3,...,3,0,0,0,9,9,18,0,0,0
1,30002,3,0,3,0,0,0,0,0,0,...,0,0,0,0,3,0,3,0,0,3
2,30003,18,24,46,0,0,3,0,6,9,...,0,0,0,0,13,20,34,9,11,16
3,30004,303,136,438,3,3,5,96,9,106,...,20,0,0,0,153,133,289,183,5,186
4,30005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,33260,37,38,73,3,0,4,4,3,8,...,6,0,0,0,38,34,76,3,4,8
3260,33261,4426,4537,8967,364,306,671,467,426,894,...,462,43,44,86,4162,4255,8420,207,234,439
3261,33262,1361,1348,2713,142,82,226,247,202,446,...,109,0,0,0,1152,1197,2351,149,154,307
3262,39494,6937,5572,12510,182,160,339,249,233,489,...,510,50,24,70,0,0,0,0,0,0


In [36]:
area_suburb_census = pd.merge(left=codes_area, right = census_16, left_index=True, right_index=True)

In [37]:
area_suburb_census

,Suburb,SSC_Code,Area,SSC_CODE_2016,Tot_P_M,Tot_P_F,Tot_P_P,Age_0_4_yr_M,Age_0_4_yr_F,Age_0_4_yr_P,...,High_yr_schl_comp_Yr_8_belw_P,High_yr_schl_comp_D_n_g_sch_M,High_yr_schl_comp_D_n_g_sch_F,High_yr_schl_comp_D_n_g_sch_P,Count_psns_occ_priv_dwgs_M,Count_psns_occ_priv_dwgs_F,Count_psns_occ_priv_dwgs_P,Count_Persons_other_dwgs_M,Count_Persons_other_dwgs_F,Count_Persons_other_dwgs_P
0,Abbeywood,30001,35.0348,30001,9,8,23,0,0,0,...,3,0,0,0,9,9,18,0,0,0
1,Abbotsford (Qld),30002,85.7638,30002,3,0,3,0,0,0,...,0,0,0,0,3,0,3,0,0,3
2,Abercorn,30003,94.2732,30003,18,24,46,0,0,3,...,0,0,0,0,13,20,34,9,11,16
3,Abergowrie,30004,306.2652,30004,303,136,438,3,3,5,...,20,0,0,0,153,133,289,183,5,186
4,Abingdon Downs,30005,5052.0282,30005,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,Yuruga,33260,108.4832,33260,37,38,73,3,0,4,...,6,0,0,0,38,34,76,3,4,8
3260,Zillmere,33261,3.6568,33261,4426,4537,8967,364,306,671,...,462,43,44,86,4162,4255,8420,207,234,439
3261,Zilzie,33262,11.6773,33262,1361,1348,2713,142,82,226,...,109,0,0,0,1152,1197,2351,149,154,307
3262,No usual address (Qld),39494,0.0000,39494,6937,5572,12510,182,160,339,...,510,50,24,70,0,0,0,0,0,0


In [38]:
# Don't need two columns with these
del area_suburb_census['SSC_CODE_2016']

In [39]:
area_suburb_census = area_suburb_census.rename(columns = {"index":"Suburb"})

In [40]:
area_suburb_census["Suburb"] = area_suburb_census["Suburb"].str.replace(' \(Qld\)','')

In [41]:
area_suburb_census["Suburb"] = area_suburb_census["Suburb"].str.upper()

In [42]:
area_suburb_census

,Suburb,SSC_Code,Area,Tot_P_M,Tot_P_F,Tot_P_P,Age_0_4_yr_M,Age_0_4_yr_F,Age_0_4_yr_P,Age_5_14_yr_M,...,High_yr_schl_comp_Yr_8_belw_P,High_yr_schl_comp_D_n_g_sch_M,High_yr_schl_comp_D_n_g_sch_F,High_yr_schl_comp_D_n_g_sch_P,Count_psns_occ_priv_dwgs_M,Count_psns_occ_priv_dwgs_F,Count_psns_occ_priv_dwgs_P,Count_Persons_other_dwgs_M,Count_Persons_other_dwgs_F,Count_Persons_other_dwgs_P
0,ABBEYWOOD,30001,35.0348,9,8,23,0,0,0,3,...,3,0,0,0,9,9,18,0,0,0
1,ABBOTSFORD,30002,85.7638,3,0,3,0,0,0,0,...,0,0,0,0,3,0,3,0,0,3
2,ABERCORN,30003,94.2732,18,24,46,0,0,3,0,...,0,0,0,0,13,20,34,9,11,16
3,ABERGOWRIE,30004,306.2652,303,136,438,3,3,5,96,...,20,0,0,0,153,133,289,183,5,186
4,ABINGDON DOWNS,30005,5052.0282,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,YURUGA,33260,108.4832,37,38,73,3,0,4,4,...,6,0,0,0,38,34,76,3,4,8
3260,ZILLMERE,33261,3.6568,4426,4537,8967,364,306,671,467,...,462,43,44,86,4162,4255,8420,207,234,439
3261,ZILZIE,33262,11.6773,1361,1348,2713,142,82,226,247,...,109,0,0,0,1152,1197,2351,149,154,307
3262,NO USUAL ADDRESS,39494,0.0000,6937,5572,12510,182,160,339,249,...,510,50,24,70,0,0,0,0,0,0


In [43]:
# created a new csv with this info
area_suburb_census.to_csv("census_suburbs_areas.csv", index=False)

In [44]:
area_suburb_census["Incidents"] = 0
area_suburb_census

,Suburb,SSC_Code,Area,Tot_P_M,Tot_P_F,Tot_P_P,Age_0_4_yr_M,Age_0_4_yr_F,Age_0_4_yr_P,Age_5_14_yr_M,...,High_yr_schl_comp_D_n_g_sch_M,High_yr_schl_comp_D_n_g_sch_F,High_yr_schl_comp_D_n_g_sch_P,Count_psns_occ_priv_dwgs_M,Count_psns_occ_priv_dwgs_F,Count_psns_occ_priv_dwgs_P,Count_Persons_other_dwgs_M,Count_Persons_other_dwgs_F,Count_Persons_other_dwgs_P,Incidents
0,ABBEYWOOD,30001,35.0348,9,8,23,0,0,0,3,...,0,0,0,9,9,18,0,0,0,0
1,ABBOTSFORD,30002,85.7638,3,0,3,0,0,0,0,...,0,0,0,3,0,3,0,0,3,0
2,ABERCORN,30003,94.2732,18,24,46,0,0,3,0,...,0,0,0,13,20,34,9,11,16,0
3,ABERGOWRIE,30004,306.2652,303,136,438,3,3,5,96,...,0,0,0,153,133,289,183,5,186,0
4,ABINGDON DOWNS,30005,5052.0282,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,YURUGA,33260,108.4832,37,38,73,3,0,4,4,...,0,0,0,38,34,76,3,4,8,0
3260,ZILLMERE,33261,3.6568,4426,4537,8967,364,306,671,467,...,43,44,86,4162,4255,8420,207,234,439,0
3261,ZILZIE,33262,11.6773,1361,1348,2713,142,82,226,247,...,0,0,0,1152,1197,2351,149,154,307,0
3262,NO USUAL ADDRESS,39494,0.0000,6937,5572,12510,182,160,339,249,...,50,24,70,0,0,0,0,0,0,0


In [45]:
bracketed = area_suburb_census[area_suburb_census["Suburb"].str.contains("[()]")]

Commented the above out for now. While this data is very nearly complete and able to be used, the head and tail of the dataframe did not reveal that many of the suburbs have bracketed ends. I'm not quite sure how to approach this right now - I think the best option may be to return to the lessons I'm doing and learn a little regex which is useful for dealing with strings.

In [46]:
bracketed["Suburb"]

23                 ALBION (BRISBANE - QLD)
24                 ALBION (RICHMOND - QLD)
38          ALLIGATOR CREEK (MACKAY - QLD)
39      ALLIGATOR CREEK (TOWNSVILLE - QLD)
88                  ASCOT (BRISBANE - QLD)
                       ...                
2817             THE GAP (MOUNT ISA - QLD)
3062             WEST END (BRISBANE - QLD)
3063           WEST END (TOWNSVILLE - QLD)
3091             WHITE ROCK (CAIRNS - QLD)
3092            WHITE ROCK (IPSWICH - QLD)
Name: Suburb, Length: 85, dtype: object

Thankfully, this simplifies things a great deal. Since we are only interested in Brisbane suburbs, the other suburbs will never come up when we are using this dataset to refer to the other. Consequently it has an easy fix.

In [47]:
area_suburb_census["Suburb"] = area_suburb_census["Suburb"].str.replace(' \(BRISBANE - QLD\)','')

In [48]:
bracketed = area_suburb_census[area_suburb_census["Suburb"].str.contains("[()]")]
bracketed["Suburb"]

24                 ALBION (RICHMOND - QLD)
38          ALLIGATOR CREEK (MACKAY - QLD)
39      ALLIGATOR CREEK (TOWNSVILLE - QLD)
89                 ASCOT (TOOWOOMBA - QLD)
263              BIRNAM (SCENIC RIM - QLD)
                       ...                
2810           THE BLUFF (TOOWOOMBA - QLD)
2817             THE GAP (MOUNT ISA - QLD)
3063           WEST END (TOWNSVILLE - QLD)
3091             WHITE ROCK (CAIRNS - QLD)
3092            WHITE ROCK (IPSWICH - QLD)
Name: Suburb, Length: 79, dtype: object

In [85]:
census_suburb_index = area_suburb_census.set_index('Suburb')

In [91]:
census_suburb_index.loc['ZILLMERE']

SSC_Code                      33261.0000
Area                              3.6568
Tot_P_M                        4426.0000
Tot_P_F                        4537.0000
Tot_P_P                        8967.0000
                                 ...    
Count_psns_occ_priv_dwgs_P     8420.0000
Count_Persons_other_dwgs_M      207.0000
Count_Persons_other_dwgs_F      234.0000
Count_Persons_other_dwgs_P      439.0000
Incidents                         0.0000
Name: ZILLMERE, Length: 111, dtype: float64

In [99]:
combined = pd.concat([census_suburb_index, suburb_counts], axis=1)

In [101]:
combined

SSC_Code                      33261.0000
Area                              3.6568
Tot_P_M                        4426.0000
Tot_P_F                        4537.0000
Tot_P_P                        8967.0000
                                 ...    
Count_Persons_other_dwgs_M      207.0000
Count_Persons_other_dwgs_F      234.0000
Count_Persons_other_dwgs_P      439.0000
Incidents                         0.0000
Suburb                          214.0000
Name: ZILLMERE, Length: 112, dtype: float64

In [104]:
combined = combined.rename({"Suburb": "Complaints"})

In [107]:
del combined['Incidents']

Finally, 